In [14]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')
#model and 
from utils import *
from tqdm import tqdm
from models import MSE,MDcR,DSE,MVBG,PCA,MVP,LPP_LE

X,labels=read_data('datasets/data sets/NGs.mat')
X = [X[0,i].T for  i  in range(X.shape[1])]
labels  = labels.squeeze()
model_MSE = MSE()
model_MDcR = MDcR()
model_DSE = DSE()
model_MVBG = MVBG(0.1,2,0.1)
model_PCA = PCA()
model_MVP =  MVP()
model_LPP_LE = LPP_LE()

res_nim = []
d_range  = range(5,20)
for d_ in tqdm(d_range): 
    nmi_list = []
    for i in range(5):
        # train_test split
        train_idx,test_idx = random_index(X[0].shape[1],0.8)
        X_train = [x[:,train_idx] for x in X]
        X_test = [x[:,test_idx] for x in X]
        y_test = labels[test_idx]
        #train
        # P = model_MSE.mse(X,gamma = 0.9,d_=d_,t=1e8,epoch=500)
        # P = model_MDcR.mdcr(X,d_=10,t=1e8,lmd=10,epoch=500)
        # P = model_PCA.dpca(X_train,d_=d_)
        # P=  model_LPP_LE.lpp(X=X_train,t=1e8,d_=d_)
        P = model_LPP_LE.le(X=X_train,d_ = d_,n_neighbors=100)
        #test
        # pred =  model_MVP.predict(X_test,0.5,1,1e8,10,d_,50)
        # pred = model_MVBG.predict(X_test,100,10,7,1e8,100)
        # pred = model_PCA.predict(X_test,P,5,False)
        pred = model_LPP_LE.predict(X_test=X_test,P=P,n_clusters=5,n_neighbors  = 100,d_ = d_,le = True)
        # criterion
        from sklearn.metrics.cluster import normalized_mutual_info_score
        nmi = normalized_mutual_info_score(y_test,pred)
        nmi_list.append(nmi)
    nmi = np.mean(nmi_list)
    res_nim.append(nmi)
    np.save('./result/NGs/le_nmi.npy',res_nim)
# res_nim
import matplotlib.pyplot as plt
plt.figure(figsize=(3,2))
plt.plot(d_range,res_nim)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


 20%|██        | 3/15 [00:11<00:45,  3.80s/it]

In [8]:
list(zip(np.load('result/NGs/dpca_nmi.npy'),d_range))


[(0.1816383430567881, 5),
 (0.11872078453312784, 6),
 (0.13184367092794938, 7),
 (0.12603075631533994, 8),
 (0.11464839574396199, 9),
 (0.13803094202707183, 10),
 (0.1302689270061492, 11),
 (0.12080955437406311, 12),
 (0.11061556382697654, 13),
 (0.13557431216257146, 14),
 (0.13978338036064605, 15),
 (0.1088345220550047, 16),
 (0.1516426693297379, 17),
 (0.19616255394006352, 18),
 (0.13462916031311964, 19)]

In [ ]:
# np.save('./result/NGs/mvbg_nmi.npy',res_nim)
# res_nmi = np.load('./result/NGs/mvbg_nmi.npy')
# res_nmi
from PIL import Image
img = Image.open('101_0108.JPG')
data = np.asarray(img)
h,w,c = data.shape
data = data.reshape(h*w,c)
data[0]

In [ ]:
#MVBG
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')
#model and 
from utils import *
from tqdm import tqdm
from models import MSE,MDcR,DSE,MVBG,PCA
from sklearn.metrics.cluster import normalized_mutual_info_score
X,labels=read_data('datasets/data sets/NGs.mat')
X = [X[0,i].T for  i  in range(X.shape[1])]
labels  = labels.squeeze()

d_range = (10,12)
model_mvbg = MVBG(0.1,2,0.1)
res_nmi = []
for d_ in d_range:
    nmi_list=[]
    for _ in range(1):
        dim_emb = model_mvbg.mvbg(X,100,d_,10)
        pred =kmeans(dim_emb,7)
        nmi_temp = normalized_mutual_info_score(labels, pred)
        nmi_list.append(nmi_temp)
    nmi = np.mean(nmi_list)
    res_nmi.append(nmi)
print(res_nmi)

In [ ]:
#DSE 單獨
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

from utils import *
from tqdm import tqdm
X,labels=read_data('datasets/data sets/YALE.mat')
X = [X[0,i].T for  i  in range(X.shape[1])]
labels  = labels.squeeze()

from models import DSE
model_DSE = DSE()
nmi_list=[]

for _ in tqdm(range(20)):

    pred = model_DSE.dse(X,7,2,500)
    from sklearn.metrics.cluster import normalized_mutual_info_score
    nmi = normalized_mutual_info_score(labels,pred)
    # print(nmi)
    nmi_list.append(nmi)
print(np.mean(nmi_list))

In [ ]:
labels.shape

In [ ]:
x=np.random.rand(3,4)
np.exp(x)/np.exp(x).sum(1).reshape(3,-1)